# Introduction
#### she changlue
20th April 2017

This project is construct to handle documents topic classification.
I use self designed algorithm to efficiently embedding the documents' topic vector and tokens' topic vector.It is emperiacally proved that this algorithm can classify the topic of different documents and tokens in a speed way.

this notebook will process as follow:
1. load library and raw corpus data
2. cut the corpus in to a list format
3. encode the tokens and corpus
4. construct model and train
5. use T-SNE to do tokens' and docs' cluster 
6. visualization
7. save the outcomes

### 1)   load library and raw corpus data

In [1]:
% matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import jieba.posseg as pseg
import jieba
import tensorflow as tf
from subprocess import check_output
print(check_output(["ls", "corpus"]).decode("utf8"))
# Any results you write to the current directory are saved as output.

custormTokens.txt
催收sample.csv
马上消费金融_jiuhui.wu_2017-04-05.csv



In [2]:
jieba.load_userdict('corpus/custormTokens.txt')  
rawdata = pd.read_csv('corpus/催收sample.csv', header=0,encoding='gbk' )

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1.NBJ\AppData\Local\Temp\jieba.cache
Loading model cost 1.593 seconds.
Prefix dict has been built succesfully.


#### corpus briefing

In [3]:
rawdata.head()

,opr_rem,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,今天下午5点472,4,NaN,NaN,NaN,NaN
1,\t告知客户电话15226088432,3,NaN,NaN,NaN,NaN
2,说是叔侄关系 转告,3,NaN,NaN,NaN,NaN
3,客户承诺今天下午五点存入184,4,NaN,NaN,NaN,NaN
4,已提醒征信影响。已告知从今天凌晨开始又会多增加75元的滞纳金。客户敷衍明天去解决,4,NaN,NaN,NaN,NaN


In [4]:
rawdata.shape

(999, 6)

In [5]:
rawdata.describe()

D:\Anaconda\lib\site-packages\numpy\lib\function_base.py:4116: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
count,999.000000,0.0,0.0,7.000000
mean,3.944945,NaN,NaN,4.000000
std,1.091033,NaN,NaN,2.160247
min,1.000000,NaN,NaN,1.000000
25%,4.000000,NaN,NaN,NaN
50%,4.000000,NaN,NaN,NaN
75%,4.000000,NaN,NaN,NaN
max,7.000000,NaN,NaN,7.000000


### 2) cut the corpus in to a list format

In [6]:
tokenCorpus  = []#corpus list of cutted tokens
rawSentences = []#raw text 
#documents    = list(rawdata[rawdata['消息目标']=='机器人']['消息内容'])#text which is send by custormers
documents    = list(rawdata['opr_rem'])#text which is send by custormers

In [7]:
# cstruct the corpus
for sentence in documents:
    sentence = sentence.replace('\t','')
    words = [pair.word for pair in pseg.lcut(sentence)]       
    if len(words)>2:
        tokenCorpus.append(words)
        rawSentences.append(sentence)

In [8]:
tokenCorpus[:3]

[['今天下午', '5', '点', '472'],
 ['告知', '客户', '电话', '15226088432'],
 ['说', '是', '叔侄', '关系', ' ', ' ', '转告', ' ']]

In [9]:
rawSentences[:5]

['今天下午5点472',
 '告知客户电话15226088432',
 '说是叔侄关系  转告 ',
 '客户承诺今天下午五点存入184',
 '已提醒征信影响。已告知从今天凌晨开始又会多增加75元的滞纳金。客户敷衍明天去解决']

### 3) encode the tokens and corpus

In [10]:
HP_miniTokenFreq = 3 #minimal tokens frequency
tokenCount = dict()  #token count
token2code = dict()  #token to code
code2token = ['inFreqTokens'] #code to token
code       = 1       #code

##### transfer token into codes

In [11]:
#get token frequency
for tokens in tokenCorpus:
    for token in tokens:
        tokenCount.setdefault(token,0)
        tokenCount[token]+=1  
#encode those tokens which have minumal frequency
for token in tokenCount:
    if tokenCount[token] > HP_miniTokenFreq:       
        token2code[token] = code
        code += 1
        code2token.append(token)
    else:
        token2code[token] = 0
#transfer the raw token corpus into encoded corpus
codeCorpus = []
for tokens in tokenCorpus:          
    codeCorpus.append([token2code[token]for token in tokens])

### 4) construct model and train

In [12]:
HP_topicNums = 8
documNums = len(codeCorpus)
tokenNums = len(code2token)
print("the training documents num:",documNums)
print("the training tokens    num:",tokenNums)

the training documents num: 992
the training tokens    num: 261


##### dense list to sparse numpy

In [13]:
tokenDocMat = np.zeros(shape=(documNums,tokenNums),dtype=np.float32)
for docIdx in range(documNums):
    tokenDocMat[docIdx][codeCorpus[docIdx]]=1
docTokenNum = tokenDocMat.sum(axis=1).reshape((-1,1))   

In [14]:
docIndxSpan = np.array(range(documNums),dtype=np.int32)

##### construct the graph

In [15]:
#embedding initial
docum2topicEmbed  = tf.Variable(tf.random_uniform([documNums,HP_topicNums], -1.0, 1.0))
token2topicWeight = tf.Variable(tf.random_uniform([HP_topicNums,tokenNums], -1.0, 1.0))

In [16]:
#placeholder
document_idx   = tf.placeholder(tf.int32, shape=[None])
document_embed = tf.nn.embedding_lookup(docum2topicEmbed , document_idx) 
multiplier     = tf.placeholder(tf.float32, shape=[None,1])
tokensInDoc    = tf.placeholder(tf.float32,shape=[None,tokenNums])

In [17]:
#forward 
with tf.name_scope("TM"):
    documentembed_Sfmx     = tf.nn.softmax(document_embed)
    token2topicWeight_Sfmx = tf.nn.softmax(token2topicWeight,dim=0)
    tokensInDoc_pred       = tf.nn.softmax(tf.matmul(documentembed_Sfmx,token2topicWeight_Sfmx))*multiplier                           

In [18]:
# use least squre loss to train model
with tf.name_scope("loss"):   
    loss = tf.reduce_mean(tf.square(tokensInDoc - tokensInDoc_pred))

In [19]:
learning_rate = 0.0001
with tf.name_scope("train"):
    optimizer   = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [20]:
#initial and save
init = tf.global_variables_initializer()
saver = tf.train.Saver()

#### training the tensorflow graph

In [33]:
#start training
n_epochs   = 1000
batch_size = 100
batch_num  = documNums//batch_size+1
assess_size= 100
with tf.Session() as sess:    
    init.run()
    for epoch in range(n_epochs):
        for batch in range(batch_num):
            #feed rawdata
            document_idx_batch = docIndxSpan [batch*batch_size:(batch+1)*batch_size]
            multiplier_batch   = docTokenNum [batch*batch_size:(batch+1)*batch_size]
            tokensInDoc_batch  = tokenDocMat [batch*batch_size:(batch+1)*batch_size]
             
            #training
            sess.run(training_op, feed_dict={document_idx : document_idx_batch,
                                             multiplier   : multiplier_batch,
                                             tokensInDoc  : tokensInDoc_batch})  
        if epoch%100==0:
            train_loss = loss.eval(feed_dict={document_idx : docIndxSpan [epoch*assess_size:(epoch+1)*assess_size],
                                              multiplier   : docTokenNum [epoch*assess_size:(epoch+1)*assess_size],
                                              tokensInDoc  : tokenDocMat [epoch*assess_size:(epoch+1)*assess_size]})
            print(train_loss)
    save_path = saver.save(sess,"tfsave/topic_model.ckpt")
    # get embedding
    documMat = documentembed_Sfmx.eval(feed_dict={document_idx : docIndxSpan})
    tokenMat = token2topicWeight_Sfmx.eval()

0.0315409
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [28]:
documMat.sum(axis=1),tokenMat.sum(axis=0)

(array([ 0.99999994,  1.        ,  1.        ,  1.        ,  0.99999988,
         1.        ,  0.99999994,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         0.99999994,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  0.99999994,  1.        ,  1.00000012,  1.        ,
         1.00000012,  1.        ,  1.00000012,  1.        ,  1.        ,
         0.99999994,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  0.99999994,  0.99999988,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  0.99999994,  1.        ,
         1.        ,  1.00000012,  1.        ,  1.        ,  1.        ,
         0.99999988,  1.        ,  1.        ,  0.99999994,  0.99999994,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         0.99999994,  0.99999994,  1.        ,  0.99999994,  1.        ,
         1.        ,  0.99999988,  0.99999994,  1. 

#### show the rough topic

In [31]:
outDF = []
keyWords = []
keySentences = []
for idx in range(HP_topicNums):    
    ords = np.argsort(-tokenMat[idx])[:10]
    tmp = ['主题'+str(idx+1)]
    tmp += [code2token[i]for i in ords]
    keyWords+=tmp
    ords = np.argsort(-documMat.T[idx])[:10]
    tmp = ['--']
    tmp +=[rawSentences[i]for i in ords]
    keySentences+=tmp
outDF = [keyWords,keySentences] 

In [32]:
pd.DataFrame(outDF).T.head(12)

,0,1
0,主题1,--
1,还清,客户姐姐，接听，说客户手机坏了，她是通过客户朋友联系到的客户，让告知客户朋友电话怎么都不愿意...
2,承诺,15968747695销售来电，走对公，333.25梁丽花2016.12.6
3,销售,其朋友接电，已告知留言转
4,后天,同事无法接听
5,被,不是赵宏国，是之前的同事，认识客户，联系不上。客户家庭情况不清楚。
6,清楚,老二18388893497未接。大姐13320494866转达客回电。小文150878765...
7,都,工人太多不清楚，也没有听过这个名字
8,换卡,18314423858 销售来电，咨询提前还清，转接客服
9,手机,明天下午三点扣款


### 5)  use T-SNE to do tokens' and docs' cluster